In [1]:
NEO4J_URI= "neo4j+s://c83b5c27.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "DVZLAV-xSNgarAF6QPOhu3vDmrIyVoR9ussQlqZravM"
NEO4J_DATABASE= "neo4j"

In [2]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD
os.environ["NEO4J_DATABASE"] = NEO4J_DATABASE

In [3]:
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant")
llm

v:\Genai_course\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026C1C1758E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026C1A17DDF0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

In [5]:
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True,allow_dangerous_requests=True, exclude_types=["Genre"])
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x0000026C7B2F4260>, cypher_generation_chain=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026C1C1758E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026C1A17DDF0>, model_name='llama-3.1-8b-in

In [6]:
chain.schema

<bound method BaseModel.schema of <class 'langchain_neo4j.chains.graph_qa.cypher.GraphCypherQAChain'>>

In [7]:
chain.graph_schema

'Node properties are the following:\nCEO {name: STRING, POB: STRING, YOB: INTEGER},Company {name: STRING},Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT},Person {name: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)'

In [12]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher Query: {query}"
)
prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher Query:",
    input_variables=["question", "schema"],
)

In [17]:
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {{name: 'Tom Hanks'}})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}, {'question': "Which directors have made mo

In [18]:
print(prompt.format(question="How many artists are there?", schema="foo"))

You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query

User input: How many artists are there?
Cypher Query: MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

User input: Which actors played in the movie Casino?
Cypher Query: MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name

User input: How many movies has Tom Hanks acted in?
Cypher Query: MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)

User input: List all the genres of the movie Schindler's List
Cypher Query: MATCH (m:Movie {title: 'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name

User input: Which actors have worked in movies from both the comedy and action genres?
Cypher Query: MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name

User input: Which directors have made movies with at least t

In [19]:
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, cypher_prompt=prompt, verbose=True, allow_dangerous_requests=True)

In [21]:
chain.invoke("actors who acted in multiple movies")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
Cypher Query: MATCH (a:Person)-[:ACTED_IN]->(m:Movie) WHERE id(m) = id(m) WITH a, COUNT(m) AS count GROUP BY a HAVING count > 1 RETURN DISTINCT a.name

This query works by connecting each actor to the movies they've acted in, then grouping them by actor. It then filters to only groups with more than one movie, and finally returns the distinct names of actors who have acted in multiple movies.

Note: The `id(m) = id(m)` part is used to force the graph to be traversed multiple times, which is necessary when counting the number of times an actor has acted in a movie. This is a common idiom in Cypher when you need to count occurrences.
Generated Cypher:
Cypher Query: MATCH (a:Person)-[:ACTED_IN]->(m:Movie) WHERE id(m) = id(m) WITH a, COUNT(m) AS count GROUP BY a HAVING count > 1 RETURN DISTINCT a.name

This query works by connecting each actor to the movies they've acted in, then grouping them by actor. It then filters to only 

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'Query': expected 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'FOREACH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 8 (offset: 7))
"Cypher Query: MATCH (a:Person)-[:ACTED_IN]->(m:Movie) WHERE id(m) = id(m) WITH a, COUNT(m) AS count GROUP BY a HAVING count > 1 RETURN DISTINCT a.name"
        ^}